In [2]:
from commonfunctions import *
import numpy as np
import cv2
import math
from scipy import ndimage
import math
import imutils
from skimage.filters import threshold_otsu, threshold_local
from skimage.morphology import binary_erosion, binary_dilation
%matplotlib qt
%load_ext autoreload
%autoreload 2

## -----------------------------------
# Auxilary Functions
## -----------------------------------

In [3]:
def digital_or_not(input_image):
    
    if '.png' in input_image.lower():
        return  1
    else:
        return  0

In [4]:
def show(img, factor=1,name="image"):
    """ 
    show an image until the escape key is pressed
    :param factor: scale factor (default 1, half size)
    """
    if factor != 1.0:
        img = cv2.resize(img, (0,0), fx=factor, fy=factor) 

    cv2.imshow(name,img)
    while(1):
        k = cv2.waitKey(0)
        if k==27:    # Esc key to stop
            break
    cv2.destroyAllWindows()

## -----------------------------------
# Image Preprocessing
## -----------------------------------

In [5]:
def gaussian(img,gaussian_kernel=5):
    
    if(len(img.shape)>2):
        img = cv2.GaussianBlur(img,(gaussian_kernel,gaussian_kernel),0)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        mid = 0.5
        mean = np.mean(img)
        gamma = math.log(mid*255)/math.log(mean)
        img = np.power(img, gamma).clip(0,255).astype(np.uint8)
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    else:
        img = cv2.GaussianBlur(img,(gaussian_kernel,gaussian_kernel),0)
    
    
    return img

In [6]:
def deskew(im, max_skew=10):

    height, width = im.shape

    # Create a grayscale image and denoise it

    im_gs = cv2.fastNlMeansDenoising(im, h=3)

    # Create an inverted B&W copy using Otsu (automatic) thresholding
    im_bw = cv2.threshold(im_gs, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

    # Detect lines in this image. Parameters here mostly arrived at by trial and error.
    lines = cv2.HoughLinesP(
        im_bw, 1, np.pi / 180, 200, minLineLength=width / 12, maxLineGap=width / 150
    )

    # Collect the angles of these lines (in radians)
    angles = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angles.append(np.arctan2(y2 - y1, x2 - x1))

    # If the majority of our lines are vertical, this is probably a landscape image
    landscape = np.sum([abs(angle) > np.pi / 4 for angle in angles]) > len(angles) / 2

    # Filter the angles to remove outliers based on max_skew
    if landscape:
        angles = [
            angle
            for angle in angles
            if np.deg2rad(90 - max_skew) < abs(angle) < np.deg2rad(90 + max_skew)
        ]
    else:
        angles = [angle for angle in angles if abs(angle) < np.deg2rad(max_skew)]

    if len(angles) < 5:
        # Insufficient data to deskew
        return im

    # Average the angles to a degree offset
    angle_deg = np.rad2deg(np.median(angles))

    # If this is landscape image, rotate the entire canvas appropriately
    if landscape:
        if angle_deg < 0:
            im = cv2.rotate(im, cv2.ROTATE_90_CLOCKWISE)
            angle_deg += 90
        elif angle_deg > 0:
            im = cv2.rotate(im, cv2.ROTATE_90_COUNTERCLOCKWISE)
            angle_deg -= 90

    # Rotate the image by the residual offset
    M = cv2.getRotationMatrix2D((width / 2, height / 2), angle_deg, 1)
    im = cv2.warpAffine(im, M, (width, height), borderMode=cv2.BORDER_REPLICATE)
    return im

In [7]:
def binarize(img,block_size=25,neighbours=7):
    if(len(img.shape)>2):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,block_size,neighbours)
    else:
        img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,block_size,neighbours)
    return img

In [8]:
def morphology(img,kernel_size=2):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(kernel_size,kernel_size))
    if(len(img.shape)>2):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        img = closing
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)      
    else:
        opening = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        closing = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        img = closing
    return img

In [60]:
def preprocess_img(img,show_steps=0,show_size=1,digital=1):
    
    if(digital != 1):
#         scale_percent = 45
#         width = int(img.shape[1] * scale_percent / 100)
#         height = int(img.shape[0] * scale_percent / 100)
#         dim = (width, height)
#         img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 
        gaussian_kernel = 15
        block_size = 95
        neighbours = 3
    else:
        gaussian_kernel = 1
        block_size = 65
        neighbours = 1
        

    # Gaussian
    img = gaussian(img,gaussian_kernel)
    if(show_steps !=0 ):
        show(img,show_size,"Gaussian Filtered")
        
    # local thresh
    img = binarize(img,block_size,neighbours)
    if(show_steps !=0 ):
        show(img,show_size,"Binarized")
        
#     # rotate
#     img = deskew(img)
#     if(show_steps !=0 ):
#         show(img,show_size,"Rotated")

    # morphology to remove noise
    if(digital != 1):
        img = morphology(img,7)
        if(show_steps !=0 ):
            show(img,show_size,"Morphology")

    return img

## -----------------------------------
# Staff Removal
## -----------------------------------

In [61]:
def run_length_encoding(img):
        
    rows = img.shape[0]
    cols = img.shape[1]

    black_runs = []
    white_runs = []
    for i in range(cols):        
        black_run = 0
        white_run = 0

        for j in range(rows):
            if (img[j][i] == 255 and black_run == 0):
                white_run += 1
            if (img[j][i] == 0 and white_run != 0):
                white_runs.append(white_run)
                white_run = 0
            if (img[j][i] == 0 and white_run == 0):
                black_run += 1
            if (img[j][i] == 255 and black_run != 0):
                black_runs.append(black_run)
                black_run = 0

    return white_runs,black_runs

In [62]:
def get_common_run(rle):
    
    counter = np.zeros(5000)
    for i in range(len(rle)):
        counter[rle[i]] += 1
        
    return np.argmax(counter)


In [63]:
def remove_lines_non_digital(img,staff_thickness,staff_space):
    rows = img.shape[0]
    cols = img.shape[1]
    
    for i in range(cols):
        j = 0
        while j < rows:
            if img[j][i] == 0 and j + staff_thickness < rows:
                if np.sum(img[j:j+staff_thickness,i]) != 0:
                    img[j:j+staff_thickness,i] = 255
                    j +=  staff_thickness -1
            j+=1
    return img

In [144]:
def get_horizontal_projection(img,show_steps=1,show_size=1):
    if(len(img.shape)>2):
        img = rgb2gray(img)
    proj = np.sum(img,axis=1).astype(int)
    
    max_line = np.amax(proj)
    staffs = proj == (max_line)
    m = np.max(proj)
    w = 500
    result = np.zeros((proj.shape[0],500))

    if show_steps == 1 :
        for row in range(img.shape[0]):
            cv2.line(result, (0,row), (int(proj[row]*w/m),row), (255,255,255), 1)
        show(result,show_size,"Horizontal Projection")
    return np.argsort(proj)

In [145]:
def identify_lines(img,show_steps=1,show_size=1):
    """ 
    Gets horizontal projection and extracts five lines from it
    :param
    show_steps: wether show steps or not 0 dont show , 1 show
    show_size : the size of the shown image a value to factor in x and y
    """
    if(len(img.shape)>2):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    staffs = get_horizontal_projection(img,show_steps,show_size)
    lines = []
    for i in range(len(staffs)):
        is_line = True
        for j in range(len(lines)):
            if(abs(staffs[i] - lines[j]) < int(img.shape[0]/30)):
                is_line = False
        if(is_line and staffs[i] - 6 > 0 and staffs[i] + 6 < img.shape[0]):
            lines.append(staffs[i])
        if(len(lines) == 5):
            break

    return img,lines

In [177]:
def remove_lines_digital(img):
    _,lines = identify_lines(img,0)
    test_value = 100
    pixels_tested = 6
    for line in lines:
        for i in range(2,img.shape[1]-2):
            sum_up = 0
            sum_down = 0
            for j in range(pixels_tested):
                sum_up += img[line-j-1][i]
                sum_down += img[line+j][i]
            if(sum_up > test_value and sum_down > test_value):
                img[line][i] = 255
                for j in range(pixels_tested):
                    img[line-j][i] = 255
                    img[line+j][i] = 255
            elif (sum_up > test_value) :
                
                img[line][i] = 255
                for j in range(pixels_tested):
                    img[line-j][i] = 255
            elif sum_down > test_value:
                img[line][i] = 255
                for j in range(pixels_tested):
                    img[line+j][i] = 255
    kernel = np.ones((3,3),np.uint8)
    img = cv2.erode(img,kernel,iterations = 1)
    return img,lines

In [180]:
def remove_staff_lines(img,show_steps=1,show_size=1,digital=1):

    white_rle,black_rle = run_length_encoding(img)
    staff_thickness = get_common_run(black_rle) + 2
    staff_space = get_common_run(white_rle) 
    img,lines = remove_lines_digital(img)
#     img = remove_lines_non_digital(img,staff_thickness,staff_space)
    if show_steps == 1 :
        show(img,show_size,"Removed Lines")
        
    return img,staff_space,staff_thickness,lines

## -----------------------------------
# Segmentation Part
## -----------------------------------

In [181]:
def get_notes(img):
    """ 
    Segements the notes
    :param
    img: the img to extract notes from
    """

    if(len(img.shape)>2):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(img)
    thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    proj = np.sum(thresh,axis=0)
    avg = np.mean(proj)
    start = 0
    end = 0
    started = False
    segments = []
    for i in range(len(proj)):
        if proj[i] > int(avg/10) and started == False:
            started = True
        if proj[i] < int(avg/10) and started == True:
            started = False
            end = i
            segments.append(img[:,start:end])
            start = i

    return segments

In [182]:
def segment_img(img,show_steps=1,show_size=1):
    segments = get_notes(img)
    if show_steps == 1 :
        for i in range(len(segments)):
            show(segments[i],show_size,"Segment")
    return segments

In [183]:
def better_segment(img):

    original = img.copy()
    blurred = cv2.GaussianBlur(img, (1, 1), 0)
    canny = cv2.Canny(blurred, 120, 255, 1)
    kernel = np.ones((5,5),np.uint8)
    dilate = cv2.dilate(canny, kernel, iterations=3)
    # Find contours
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    # Iterate thorugh contours and filter for ROI
    image_number = 0
    order_list = []
    images = []
    for c in cnts:
        x,y,w,h = cv2.boundingRect(c)
        cv2.rectangle(img, (x, y), (x + w, y + h), (36,255,12), 2)
        ROI = original[y:y+h, x:x+w]
        images.append(ROI)
        order_list.append(x)
        image_number += 1


    segments = [x for _,x in sorted(zip(order_list,images))]
    return segments

## -----------------------------------
# Calssification
## -----------------------------------

In [184]:
def check_full_symmetry(image):
    notImg = cv2.bitwise_not(image)
    avg = np.sum(notImg)/4
    thresh = avg - int(avg/10)
    rows = notImg.shape[0]
    cols = notImg.shape[1]
    q1 = np.sum(notImg[0:int(rows/2),0:int(cols/2)])
    q2 = np.sum(notImg[0:int(rows/2),int(cols/2):cols])
    q3 = np.sum(notImg[int(rows/2):rows,0:int(cols/2)])
    q4 = np.sum(notImg[int(rows/2):rows,int(cols/2):cols])
    #print("Q1 is " ,q1, " and Q2 is ", q2, " and Q3 is ", q3, " and Q4 is ", q4)
    #print("Avg is ", avg)
    if q1  >= thresh and q2 >= thresh and q3 >= thresh and q4 >= thresh:
        return True
    else: 
        return False

In [185]:
def get_v_lines(img): #This function is for use in classifications only
    
    gray = img

    blur_gray = gray
    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)
    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 10  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = img.shape[0]-int(img.shape[0]/8)  # minimum number of pixels making up a line
    max_line_gap = img.shape[0]-int(img.shape[0]/8)  # maximum gap in pixels between connectable line segments
    line_image = np.copy(img) * 0  # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)
    if lines is None:
        return 0
    return len(lines)


In [186]:
def check_half_symmetry(image):
    notImg = cv2.bitwise_not(image)
    rows = notImg.shape[0]
    cols = notImg.shape[1]
    topSum =  np.sum(notImg[0:int(rows/2),:])
    bottomSum = np.sum(notImg[int(rows/2):rows,: ])
    leftSum =np.sum(notImg[:, 0:int(cols/2)])
    rightSum = np.sum(notImg[:, int(cols/2): cols]) 
    topSum = np.int64(topSum)
    bottomSum = np.int64(bottomSum)
    leftSum = np.int64(leftSum)
    rightSum = np.int64(rightSum)
    if abs(leftSum - rightSum) <= int(leftSum/10):
        
        if abs(topSum - bottomSum) > int(bottomSum/5):
            return "double flat"
        else:
            return "natural"
    
    else:
        return "flat"

In [187]:
def crop_segment(segment):
    mask_inv = cv2.bitwise_not(np.copy(segment))
    ver_sum = np.sum(mask_inv,axis=1)
    v_start = 0
    v_end = 0
    for i in range(len(ver_sum)):
        if(ver_sum[i] > 0 and v_start ==0):
            v_start = i
        if(ver_sum[i] == 0 and v_start != 0):
            v_end = i
            break
    if(v_end == 0):
        v_end = len(ver_sum) - 1
    
    hor_sum = np.sum(mask_inv,axis=0)
    h_start = 0
    h_end = 0
    for i in range(len(hor_sum)):
        if(hor_sum[i] > 0 and h_start ==0):
            h_start = i
        if(hor_sum[i] == 0 and h_start != 0):
            h_end = i
            break
    if(h_end == 0):
        h_end = len(hor_sum) - 1

    return segment[v_start:v_end,h_start:h_end]

In [188]:
def classify(segment, line_space):
        rows = segment.shape[0]
        mask_inv = cv2.bitwise_not(np.copy(segment))
        sum_ver = np.sum(mask_inv,axis=1)
        if (np.count_nonzero(sum_ver) >= line_space*4):
            
            #TODO: classify note here
            return "Note"
            pass
        else:
            segment = crop_segment(segment)
            if (check_full_symmetry(segment)):
                if get_v_lines(segment) > 1 :
                    return "#"
                else:
                    return "##"
            else:
                classification = check_half_symmetry(segment)
                if (classification  == None):
                    return None
                else:
                    return classification



## -----------------------------------
# Main Code
## -----------------------------------

In [189]:
input_image = '/home/kamel/Desktop/Image/Project/OMR_IP/input/02.PNG'
img = cv2.imread(input_image) 

digital = digital_or_not(input_image)
show_steps = 1
show_size = 1


img = preprocess_img(img,show_steps,show_size,digital)
img,staff_space,staff_thickness,lines = remove_staff_lines(img,show_steps,show_size)
# segments = segment_img(img,show_steps,show_size)
segments = segment_img(img,0)
for i in range(len(segments)):
    show(segments[i],1,"Segment " + str(i+1))
    print(classify(segments[i], staff_space))



Note
Note
Note
flat
Note
Note
#
Note
Note
Note
double flat
Note
natural
Note
flat
flat
Note
Note
Note
Note


## -----------------------------------
# Test Area
## -----------------------------------

In [58]:
def get_h_lines(img):
    img = cv2.imread('/home/kamel/Desktop/Image/Project/OMR_IP/input/05.PNG')
        
    gray = preprocess_img(img,0,show_size,0)

    low_threshold = 50
    high_threshold = 150
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    
    rho = 1  # distance resolution in pixels of the Hough grid
    theta = np.pi / 180  # angular resolution in radians of the Hough grid
    threshold = 15  # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 500  # minimum number of pixels making up a line
    max_line_gap = 500  # maximum gap in pixels between connectable line segments
    line_image = np.copy(img) * 0  # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]),
                        min_line_length, max_line_gap)
    max_y = []
    max_x = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),5)    
            # Draw the lines on the  image
    lines_edges = cv2.addWeighted(img, 0.8, line_image, 1, 0)
    

    return line_image

img = get_h_lines(None)
show(img,1)

In [203]:
def rotate_image(img,show_steps=1,show_size=1):
    
    original = cv2.imread('/home/kamel/Desktop/Image/Project/OMR_IP/input/19.jpg')
    color = ('b','g','r')
#     for i,col in enumerate(color):
#         histr = cv2.calcHist([original],[i],None,[256],[0,256])
#         plt.plot(histr,color = col)
#         plt.xlim([0,256])
#     plt.show()
    original[original[:,:,0]<120] = 0
    original[original[:,:,0]>210] = 0
    original[original[:,:,1]<100] = 0
    original[original[:,:,1]>210] = 0
    original[original[:,:,2]<120] = 0
    original[original[:,:,2]>210] = 0

    show(original,show_size,"TEST")
    img = preprocess_img(original,0,show_size,0)
    if show_steps == 1 : 
        show(img,0.2,"Blurred")

    canny = cv2.Canny(img, 200, 255, 1)

    if show_steps == 1 :
        show(canny,show_size,"Canny")
    kernel = np.ones((25,25),np.uint8)
    dilate = cv2.dilate(canny, kernel, iterations=1)
    show(dilate,show_size)
    contours, hierarchy = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    if len(contours) != 0:
        cv2.drawContours(img, contours, -1, 255, 3)
        c = max(contours, key = cv2.contourArea)
        rect = cv2.minAreaRect(c)
        box = cv2.boxPoints(rect)
        box = np.int0(box)

        cv2.drawContours(img,[box],0,(0,0,255),2)
        
    rows,cols = img.shape[:2]
    [vx,vy,x,y] = cv2.fitLine(c, cv2.DIST_L2,0,0.01,0.01)
    lefty = int((-x*vy/vx) + y)
    righty = int(((cols-x)*vy/vx)+y)
    cv2.line(img,(cols-1,righty),(0,lefty),(0,255,0),2)        

    if show_steps == 1 :
        show(img,show_size,"Contour")

    tr = (box[3][0],box[3][1])
    tl = (box[0][0],box[0][1])
    bl = (box[1][0],box[1][1])
    br = (box[2][0],box[2][1])

    corner_points_array = np.float32([tl,tr,br,bl])    
    cv2.circle(original,tl,10,(255,255,0),thickness=7)
    cv2.circle(original,tr,10,(255,255,0),thickness=7)
    cv2.circle(original,bl,10,(255,255,0),thickness=7)
    cv2.circle(original,br,10,(255,255,0),thickness=7)
    show(original,show_size,"The circled")

    width = original.shape[0]
    height = original.shape[1]

    # Create an array with the parameters (the dimensions) required to build the matrix
    imgTl = [0,0]
    imgTr = [0,width]
    imgBr = [height,width]
    imgBl = [height,0]
    img_params = np.float32([imgTl,imgTr,imgBr,imgBl])

    # Compute and return the transformation matrix
    matrix = cv2.getPerspectiveTransform(corner_points_array,img_params)
    img_transformed = cv2.warpPerspective(original,matrix,(width,height))

    if show_steps == 1 :
        show(img_transformed,show_size)
    
    return img_transformed

    
# img = get_h_lines(None)
# if show_steps == 1 :
#     show(img,0.2,"HLine")
img = rotate_image(None,show_steps=1,show_size=0.2)
    

## -----------------------------------
# Not Needed
## -----------------------------------